# This code detects song in the .wav files inside of a folder.

## This cell of code takes in a folder, then generates 1-minute long .wav files by combining and splitting up recordings

In [1]:
import os
import numpy as np
from scipy.io import wavfile
import json

def read_audio_files(folder_path):
    audio_data = []
    samplerates = []
    file_details = []  # List to store details for each file

    for file_name in os.listdir(folder_path):
        if file_name.endswith('.wav'):
            file_path = os.path.join(folder_path, file_name)
            try:
                samplerate, data = wavfile.read(file_path)
                if data.ndim > 1:  # Convert to mono if stereo
                    data = data.mean(axis=1)
                audio_data.append(data)
                samplerates.append(samplerate)
                file_details.append({'file_name': file_name, 'samples': len(data)})
            except Exception as e:
                print(f"Error processing {file_name}: {e}")
                continue

    assert all(x == samplerates[0] for x in samplerates), "Sample rates differ among files"
    return audio_data, samplerates[0], file_details

def concatenate_to_minute_segments(audio_data, samplerate, file_details, target_duration_minutes=1):
    target_duration_samples = target_duration_minutes * 60 * samplerate
    concatenated_data = np.concatenate(audio_data)

    num_segments = concatenated_data.size // target_duration_samples
    total_samples_needed = num_segments * target_duration_samples
    concatenated_data = concatenated_data[:total_samples_needed]

    segments = []
    segment_file_lists = []
    current_sample_index = 0  # Track which file we are in
    cumulative_file_end = 0   # Track the end sample of current file globally

    for i in range(num_segments):
        start_index = i * target_duration_samples
        end_index = start_index + target_duration_samples
        segment = concatenated_data[start_index:end_index]
        segments.append(segment)

        files_in_segment = []
        seg_start = start_index
        seg_end = end_index
        seg_relative_offset = 0  # Track where we are in the segment

        file_start_sample = 0  # Absolute start sample of the file

        for j, file_info in enumerate(file_details):
            file_length = file_info['samples']
            file_end_sample = file_start_sample + file_length

            # Check for overlap with segment
            overlap_start = max(file_start_sample, seg_start)
            overlap_end = min(file_end_sample, seg_end)

            if overlap_end > overlap_start:
                rel_start = overlap_start - seg_start
                rel_end = overlap_end - seg_start
                files_in_segment.append({
                    'file_name': file_info['file_name'],
                    'start_sample': int(rel_start),
                    'end_sample': int(rel_end)
                })

            file_start_sample = file_end_sample
            if file_start_sample >= seg_end:
                break

        segment_file_lists.append(files_in_segment)

    return segments, segment_file_lists

# === Run the processing ===
folder_path = '/Volumes/my_own_SSD/UO_stuff/nerve_transections/USA5201/60'  # Replace with your folder path
audio_data, samplerate, file_details = read_audio_files(folder_path)

# Create the new folder for segmented files
output_folder = os.path.join(folder_path, 'one_minute_segments')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Generate segments and detailed mappings
segments, segment_file_lists = concatenate_to_minute_segments(audio_data, samplerate, file_details)

# Save segmented .wav files and detailed file-to-segment mappings
json_data = {}
for i, (segment, file_list) in enumerate(zip(segments, segment_file_lists)):
    output_path = os.path.join(output_folder, f"one_minute_segment_{i+1}.wav")
    wavfile.write(output_path, samplerate, segment.astype(np.int16))
    print(f"Saved: {output_path}")
    json_data[f"one_minute_segment_{i+1}.wav"] = file_list

# Save file segment mapping to JSON
json_output_path = os.path.join(output_folder, 'file_lists.json')
with open(json_output_path, 'w') as f:
    json.dump(json_data, f, indent=4)
    print(f"File list with sample ranges saved to {json_output_path}")

Saved: /Volumes/my_own_SSD/UO_stuff/nerve_transections/USA5201/60/one_minute_segments/one_minute_segment_1.wav
Saved: /Volumes/my_own_SSD/UO_stuff/nerve_transections/USA5201/60/one_minute_segments/one_minute_segment_2.wav
Saved: /Volumes/my_own_SSD/UO_stuff/nerve_transections/USA5201/60/one_minute_segments/one_minute_segment_3.wav
Saved: /Volumes/my_own_SSD/UO_stuff/nerve_transections/USA5201/60/one_minute_segments/one_minute_segment_4.wav
Saved: /Volumes/my_own_SSD/UO_stuff/nerve_transections/USA5201/60/one_minute_segments/one_minute_segment_5.wav
Saved: /Volumes/my_own_SSD/UO_stuff/nerve_transections/USA5201/60/one_minute_segments/one_minute_segment_6.wav
Saved: /Volumes/my_own_SSD/UO_stuff/nerve_transections/USA5201/60/one_minute_segments/one_minute_segment_7.wav
Saved: /Volumes/my_own_SSD/UO_stuff/nerve_transections/USA5201/60/one_minute_segments/one_minute_segment_8.wav
Saved: /Volumes/my_own_SSD/UO_stuff/nerve_transections/USA5201/60/one_minute_segments/one_minute_segment_9.wav
S

# This code goes through each of the 1-minute long .wav files, detects song, then saves images of spectrograms with the highlighted detected song

## Reduce spacing between each plot

In [2]:
import numpy as np
import os
from pathlib import Path
from scipy.io import wavfile
from scipy.signal import spectrogram, windows, ellip, filtfilt
from scipy.ndimage import gaussian_filter1d
import matplotlib.pyplot as plt
import tkinter as tk

def get_screen_resolution():
    root = tk.Tk()
    root.withdraw()
    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()
    root.destroy()
    return screen_width / 100, screen_height / 100  # inches

width_inches, height_inches = get_screen_resolution()

def process_wav_file(file_path, amplitude_folder, spectrogram_folder, segment_duration=10, low_cut=500, high_cut=8000, sigma=100, threshold=0.05):
    try:
        samplerate, data = wavfile.read(file_path)
        if data.ndim > 1:
            data = data.mean(axis=1)

        nyquist = samplerate / 2
        wp = [low_cut / nyquist, high_cut / nyquist]
        b, a = ellip(5, 0.2, 40, wp, btype='band')
        data = filtfilt(b, a, data)

        duration_seconds = data.shape[0] / samplerate
        segment_length_samples = int(segment_duration * samplerate)
        num_segments = int(np.ceil(duration_seconds / segment_duration))

        base_name = Path(file_path).stem
        amplitude_fig_path = os.path.join(amplitude_folder, f"{base_name}_amplitude_trace.png")
        spectrogram_fig_path = os.path.join(spectrogram_folder, f"{base_name}_detected_song_spectrogram.png")

        fig1, axs_amp = plt.subplots(num_segments, 1, figsize=(width_inches, height_inches), sharex=True, gridspec_kw={'hspace': 0.0})
        if num_segments == 1:
            axs_amp = [axs_amp]

        fig2, axs_spec = plt.subplots(num_segments, 1, figsize=(width_inches, height_inches), sharex=True, gridspec_kw={'hspace': 0.0})
        if num_segments == 1:
            axs_spec = [axs_spec]

        for i in range(num_segments):
            start_sample = i * segment_length_samples
            end_sample = start_sample + segment_length_samples
            segment_data = np.zeros(segment_length_samples, dtype=data.dtype)
            if start_sample < data.shape[0]:
                segment_data[:max(0, min(segment_length_samples, data.shape[0] - start_sample))] = data[start_sample:end_sample]

            f, t, Sxx = spectrogram(
                segment_data,
                fs=samplerate,
                window=windows.gaussian(2048, std=2048/8),
                nperseg=2048,
                noverlap=(2048 - 119)
            )

            Sxx_log = 10 * np.log10(Sxx + np.finfo(float).eps)
            Sxx_log_clipped = np.clip(Sxx_log, a_min=-2, a_max=None)
            Sxx_log_normalized = (Sxx_log_clipped - np.min(Sxx_log_clipped)) / (np.max(Sxx_log_clipped) - np.min(Sxx_log_clipped))

            freq_mask = (f >= low_cut) & (f <= high_cut)
            Sxx_band = Sxx[freq_mask, :]
            amplitude_trace = np.sum(Sxx_band, axis=0)
            smoothed_trace = gaussian_filter1d(amplitude_trace, sigma=sigma)
            is_song = smoothed_trace > threshold

            axs_amp[i].plot(t, amplitude_trace, color='gray', alpha=0.5, label='Original')
            axs_amp[i].plot(t, smoothed_trace, color='black', label=f'Smoothed (σ={sigma})')
            axs_amp[i].axhline(threshold, color='red', linestyle='--', linewidth=1, label=f'Threshold = {threshold}')
            axs_amp[i].fill_between(t, smoothed_trace, threshold, where=is_song, interpolate=True,
                                    color='gold', alpha=0.4, label='Detected Song' if i == 0 else None)
            axs_amp[i].set_ylabel('Amplitude')
            if i == 0:
                axs_amp[i].legend(loc='upper right', fontsize=8)
            if i == num_segments - 1:
                axs_amp[i].set_xlabel('Time [sec]')
                axs_amp[i].set_xticks(np.linspace(0, segment_duration, 5))

            axs_spec[i].imshow(Sxx_log_normalized, aspect='auto', origin='lower',
                               extent=[0, segment_duration, f.min(), f.max()], cmap='binary')
            axs_spec[i].fill_between(t, f.min(), f.max(), where=is_song, color='gold', alpha=0.3)
            axs_spec[i].set_ylabel('Freq [Hz]')
            if i == num_segments - 1:
                axs_spec[i].set_xlabel('Time [sec]')
                axs_spec[i].set_xticks(np.linspace(0, segment_duration, 5))

            print(f"Processed segment {i+1}/{num_segments} for file {base_name} – Song {'🎵 Detected' if np.any(is_song) else '⚪ Not Detected'}")

        fig1.suptitle(f'{base_name} – Amplitude Traces with Song Detection', fontsize=14)
        fig2.suptitle(f'{base_name} – Spectrogram with Detected Song', fontsize=14)

        fig1.tight_layout()
        fig2.tight_layout()

        fig1.savefig(amplitude_fig_path, dpi=300)
        fig2.savefig(spectrogram_fig_path, dpi=300)
        plt.close(fig1)
        plt.close(fig2)

        print(f"✅ Saved: {amplitude_fig_path}")
        print(f"✅ Saved: {spectrogram_fig_path}\n")

    except Exception as e:
        print(f"❌ Error processing {file_path}: {e}")

def batch_process_folder(folder_path, segment_duration=10, sigma=100, threshold=0.05):
    output_folder = os.path.join(folder_path, "song_detection_outputs")
    amplitude_folder = os.path.join(output_folder, "amplitude_traces")
    spectrogram_folder = os.path.join(output_folder, "detected_song_spectrograms")
    os.makedirs(amplitude_folder, exist_ok=True)
    os.makedirs(spectrogram_folder, exist_ok=True)

    wav_files = [f for f in Path(folder_path).glob("*.wav")]
    if not wav_files:
        print("No .wav files found in the selected folder.")
        return

    print(f"\n📂 Processing {len(wav_files)} files in: {folder_path}\n")
    for wav_file in wav_files:
        process_wav_file(wav_file, amplitude_folder, spectrogram_folder,
                         segment_duration=segment_duration,
                         sigma=sigma,
                         threshold=threshold)

# === USER INPUT ===
#folder_path = '/Users/mirandahulsey-vincent/Documents/allPythonCode/BYOD_class/data_inputs/USA5483_sample_songs/216_sample'
folder_path = folder_path + '/one_minute_segments/'
batch_process_folder(folder_path, sigma=70, threshold=50)



📂 Processing 76 files in: /Volumes/my_own_SSD/UO_stuff/nerve_transections/USA5201/60/one_minute_segments/

Processed segment 1/6 for file one_minute_segment_1 – Song 🎵 Detected
Processed segment 2/6 for file one_minute_segment_1 – Song 🎵 Detected
Processed segment 3/6 for file one_minute_segment_1 – Song 🎵 Detected
Processed segment 4/6 for file one_minute_segment_1 – Song 🎵 Detected
Processed segment 5/6 for file one_minute_segment_1 – Song 🎵 Detected
Processed segment 6/6 for file one_minute_segment_1 – Song 🎵 Detected
✅ Saved: /Volumes/my_own_SSD/UO_stuff/nerve_transections/USA5201/60/one_minute_segments/song_detection_outputs/amplitude_traces/one_minute_segment_1_amplitude_trace.png
✅ Saved: /Volumes/my_own_SSD/UO_stuff/nerve_transections/USA5201/60/one_minute_segments/song_detection_outputs/detected_song_spectrograms/one_minute_segment_1_detected_song_spectrogram.png

Processed segment 1/6 for file one_minute_segment_2 – Song 🎵 Detected
Processed segment 2/6 for file one_minute_s

## Increase constrast on spectrograms

In [3]:
# import numpy as np
# import os
# from pathlib import Path
# from scipy.io import wavfile
# from scipy.signal import spectrogram, windows, ellip, filtfilt
# from scipy.ndimage import gaussian_filter1d
# import matplotlib.pyplot as plt
# import tkinter as tk

# def get_screen_resolution():
#     root = tk.Tk()
#     root.withdraw()
#     screen_width = root.winfo_screenwidth()
#     screen_height = root.winfo_screenheight()
#     root.destroy()
#     return screen_width / 100, screen_height / 100  # inches

# width_inches, height_inches = get_screen_resolution()

# def process_wav_file(file_path, amplitude_folder, spectrogram_folder, segment_duration=10, low_cut=500, high_cut=8000, sigma=100, threshold=0.05):
#     try:
#         samplerate, data = wavfile.read(file_path)
#         if data.ndim > 1:
#             data = data.mean(axis=1)

#         nyquist = samplerate / 2
#         wp = [low_cut / nyquist, high_cut / nyquist]
#         b, a = ellip(5, 0.2, 40, wp, btype='band')
#         data = filtfilt(b, a, data)

#         duration_seconds = data.shape[0] / samplerate
#         segment_length_samples = int(segment_duration * samplerate)
#         num_segments = int(np.ceil(duration_seconds / segment_duration))

#         base_name = Path(file_path).stem
#         amplitude_fig_path = os.path.join(amplitude_folder, f"{base_name}_amplitude_trace.png")
#         spectrogram_fig_path = os.path.join(spectrogram_folder, f"{base_name}_detected_song_spectrogram.png")

#         fig1, axs_amp = plt.subplots(num_segments, 1, figsize=(width_inches, height_inches), sharex=True, gridspec_kw={'hspace': 0.0})
#         if num_segments == 1:
#             axs_amp = [axs_amp]

#         fig2, axs_spec = plt.subplots(num_segments, 1, figsize=(width_inches, height_inches), sharex=True, gridspec_kw={'hspace': 0.0})
#         if num_segments == 1:
#             axs_spec = [axs_spec]

#         for i in range(num_segments):
#             start_sample = i * segment_length_samples
#             end_sample = start_sample + segment_length_samples
#             segment_data = np.zeros(segment_length_samples, dtype=data.dtype)
#             if start_sample < data.shape[0]:
#                 segment_data[:max(0, min(segment_length_samples, data.shape[0] - start_sample))] = data[start_sample:end_sample]

#             f, t, Sxx = spectrogram(
#                 segment_data,
#                 fs=samplerate,
#                 window=windows.gaussian(2048, std=2048/8),
#                 nperseg=2048,
#                 noverlap=(2048 - 119)
#             )

#             Sxx_log = 10 * np.log10(Sxx + np.finfo(float).eps)
#             Sxx_log_clipped = np.clip(Sxx_log, a_min=-5, a_max=None)
#             Sxx_log_normalized = (Sxx_log_clipped - np.min(Sxx_log_clipped)) / (np.max(Sxx_log_clipped) - np.min(Sxx_log_clipped))
#             Sxx_log_normalized = np.power(Sxx_log_normalized, 0.7)  # Enhance contrast

#             freq_mask = (f >= low_cut) & (f <= high_cut)
#             Sxx_band = Sxx[freq_mask, :]
#             amplitude_trace = np.sum(Sxx_band, axis=0)
#             smoothed_trace = gaussian_filter1d(amplitude_trace, sigma=sigma)
#             is_song = smoothed_trace > threshold

#             axs_amp[i].plot(t, amplitude_trace, color='gray', alpha=0.5, label='Original')
#             axs_amp[i].plot(t, smoothed_trace, color='black', label=f'Smoothed (σ={sigma})')
#             axs_amp[i].axhline(threshold, color='red', linestyle='--', linewidth=1, label=f'Threshold = {threshold}')
#             axs_amp[i].fill_between(t, smoothed_trace, threshold, where=is_song, interpolate=True,
#                                     color='gold', alpha=0.4, label='Detected Song' if i == 0 else None)
#             axs_amp[i].set_ylabel('Amplitude')
#             if i == 0:
#                 axs_amp[i].legend(loc='upper right', fontsize=8)
#             if i == num_segments - 1:
#                 axs_amp[i].set_xlabel('Time [sec]')
#                 axs_amp[i].set_xticks(np.linspace(0, segment_duration, 5))

#             axs_spec[i].imshow(Sxx_log_normalized, aspect='auto', origin='lower',
#                                extent=[0, segment_duration, f.min(), f.max()], cmap='binary')
#             axs_spec[i].fill_between(t, f.min(), f.max(), where=is_song, color='gold', alpha=0.3)
#             axs_spec[i].set_ylabel('Freq [Hz]')
#             if i == num_segments - 1:
#                 axs_spec[i].set_xlabel('Time [sec]')
#                 axs_spec[i].set_xticks(np.linspace(0, segment_duration, 5))

#             print(f"Processed segment {i+1}/{num_segments} for file {base_name} – Song {'🎵 Detected' if np.any(is_song) else '⚪ Not Detected'}")

#         fig1.suptitle(f'{base_name} – Amplitude Traces with Song Detection', fontsize=14)
#         fig2.suptitle(f'{base_name} – Spectrogram with Detected Song', fontsize=14)

#         fig1.tight_layout()
#         fig2.tight_layout()

#         fig1.savefig(amplitude_fig_path, dpi=300)
#         fig2.savefig(spectrogram_fig_path, dpi=300)
#         plt.close(fig1)
#         plt.close(fig2)

#         print(f"✅ Saved: {amplitude_fig_path}")
#         print(f"✅ Saved: {spectrogram_fig_path}\n")

#     except Exception as e:
#         print(f"❌ Error processing {file_path}: {e}")

# def batch_process_folder(folder_path, segment_duration=10, sigma=100, threshold=0.05):
#     output_folder = os.path.join(folder_path, "song_detection_outputs")
#     amplitude_folder = os.path.join(output_folder, "amplitude_traces")
#     spectrogram_folder = os.path.join(output_folder, "detected_song_spectrograms")
#     os.makedirs(amplitude_folder, exist_ok=True)
#     os.makedirs(spectrogram_folder, exist_ok=True)

#     wav_files = [f for f in Path(folder_path).glob("*.wav")]
#     if not wav_files:
#         print("No .wav files found in the selected folder.")
#         return

#     print(f"\n📂 Processing {len(wav_files)} files in: {folder_path}\n")
#     for wav_file in wav_files:
#         process_wav_file(wav_file, amplitude_folder, spectrogram_folder,
#                          segment_duration=segment_duration,
#                          sigma=sigma,
#                          threshold=threshold)

# # === USER INPUT ===
# folder_path = folder_path + '/one_minute_segments/'
# batch_process_folder(folder_path, sigma=50, threshold=80)

## Add a red vertical line demarcating the beginning and end of each recording file in the spectrogram

In [4]:
import numpy as np
import os
from pathlib import Path
from scipy.io import wavfile
from scipy.signal import spectrogram, windows, ellip, filtfilt
from scipy.ndimage import gaussian_filter1d
import matplotlib.pyplot as plt
import tkinter as tk
import json

def get_screen_resolution():
    root = tk.Tk()
    root.withdraw()
    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()
    root.destroy()
    return screen_width / 100, screen_height / 100  # inches

width_inches, height_inches = get_screen_resolution()

# === Load JSON file once
with open(folder_path + 'file_lists.json', 'r') as f:
    file_lists = json.load(f)

def process_wav_file(file_path, amplitude_folder, spectrogram_folder, segment_duration=10, low_cut=500, high_cut=8000, sigma=100, threshold=0.05):
    try:
        base_name = Path(file_path).stem
        samplerate, data = wavfile.read(file_path)
        if data.ndim > 1:
            data = data.mean(axis=1)

        nyquist = samplerate / 2
        wp = [low_cut / nyquist, high_cut / nyquist]
        b, a = ellip(5, 0.2, 40, wp, btype='band')
        data = filtfilt(b, a, data)

        duration_seconds = data.shape[0] / samplerate
        segment_length_samples = int(segment_duration * samplerate)
        num_segments = int(np.ceil(duration_seconds / segment_duration))

        amplitude_fig_path = os.path.join(amplitude_folder, f"{base_name}_amplitude_trace.png")
        spectrogram_fig_path = os.path.join(spectrogram_folder, f"{base_name}_detected_song_spectrogram.png")

        fig1, axs_amp = plt.subplots(num_segments, 1, figsize=(width_inches, height_inches), sharex=True, gridspec_kw={'hspace': 0.0})
        fig2, axs_spec = plt.subplots(num_segments, 1, figsize=(width_inches, height_inches), sharex=True, gridspec_kw={'hspace': 0.0})
        if num_segments == 1:
            axs_amp = [axs_amp]
            axs_spec = [axs_spec]

        # Get red line timestamps
        red_lines_sec = []
        if f"{base_name}.wav" in file_lists:
            red_lines_sec = [entry["start_sample"] / samplerate for entry in file_lists[f"{base_name}.wav"]]

        for i in range(num_segments):
            start_sample = i * segment_length_samples
            end_sample = start_sample + segment_length_samples
            segment_data = np.zeros(segment_length_samples, dtype=data.dtype)
            if start_sample < data.shape[0]:
                segment_data[:max(0, min(segment_length_samples, data.shape[0] - start_sample))] = data[start_sample:end_sample]

            f, t, Sxx = spectrogram(
                segment_data,
                fs=samplerate,
                window=windows.gaussian(2048, std=2048/8),
                nperseg=2048,
                noverlap=(2048 - 119)
            )

            Sxx_log = 10 * np.log10(Sxx + np.finfo(float).eps)
            Sxx_log_clipped = np.clip(Sxx_log, a_min=3, a_max=None)
            Sxx_log_normalized = (Sxx_log_clipped - np.min(Sxx_log_clipped)) / (np.max(Sxx_log_clipped) - np.min(Sxx_log_clipped))
            Sxx_log_normalized = np.power(Sxx_log_normalized, 0.7)

            freq_mask = (f >= low_cut) & (f <= high_cut)
            Sxx_band = Sxx[freq_mask, :]
            amplitude_trace = np.sum(Sxx_band, axis=0)
            smoothed_trace = gaussian_filter1d(amplitude_trace, sigma=sigma)
            is_song = smoothed_trace > threshold

            axs_amp[i].plot(t, amplitude_trace, color='gray', alpha=0.5, label='Original')
            axs_amp[i].plot(t, smoothed_trace, color='black', label=f'Smoothed (σ={sigma})')
            axs_amp[i].axhline(threshold, color='red', linestyle='--', linewidth=1, label=f'Threshold = {threshold}')
            axs_amp[i].fill_between(t, smoothed_trace, threshold, where=is_song, interpolate=True,
                                    color='gold', alpha=0.4, label='Detected Song' if i == 0 else None)

            for x in red_lines_sec:
                if start_sample / samplerate <= x < end_sample / samplerate:
                    axs_amp[i].axvline(x - (start_sample / samplerate), color='red', linestyle='-', linewidth=1)

            axs_amp[i].set_ylabel('Amplitude')
            if i == 0:
                axs_amp[i].legend(loc='upper right', fontsize=8)
            if i == num_segments - 1:
                axs_amp[i].set_xlabel('Time [sec]')
                axs_amp[i].set_xticks(np.linspace(0, segment_duration, 5))

            axs_spec[i].imshow(Sxx_log_normalized, aspect='auto', origin='lower',
                               extent=[0, segment_duration, f.min(), f.max()], cmap='binary')
            axs_spec[i].fill_between(t, f.min(), f.max(), where=is_song, color='gold', alpha=0.3)

            for x in red_lines_sec:
                if start_sample / samplerate <= x < end_sample / samplerate:
                    axs_spec[i].axvline(x - (start_sample / samplerate), color='red', linestyle='-', linewidth=1)

            axs_spec[i].set_ylabel('Freq [Hz]')
            if i == num_segments - 1:
                axs_spec[i].set_xlabel('Time [sec]')
                axs_spec[i].set_xticks(np.linspace(0, segment_duration, 5))

            print(f"Processed segment {i+1}/{num_segments} for file {base_name} – Song {'🎵 Detected' if np.any(is_song) else '⚪ Not Detected'}")

        fig1.suptitle(f'{base_name} – Amplitude Traces with Song Detection', fontsize=14)
        fig2.suptitle(f'{base_name} – Spectrogram with Detected Song', fontsize=14)

        fig1.tight_layout()
        fig2.tight_layout()

        fig1.savefig(amplitude_fig_path, dpi=300)
        fig2.savefig(spectrogram_fig_path, dpi=300)
        plt.close(fig1)
        plt.close(fig2)

        print(f"✅ Saved: {amplitude_fig_path}")
        print(f"✅ Saved: {spectrogram_fig_path}\n")

    except Exception as e:
        print(f"❌ Error processing {file_path}: {e}")

def batch_process_folder(folder_path, segment_duration=10, sigma=100, threshold=0.05):
    output_folder = os.path.join(folder_path, "song_detection_outputs")
    amplitude_folder = os.path.join(output_folder, "amplitude_traces")
    spectrogram_folder = os.path.join(output_folder, "detected_song_spectrograms")
    os.makedirs(amplitude_folder, exist_ok=True)
    os.makedirs(spectrogram_folder, exist_ok=True)

    wav_files = [f for f in Path(folder_path).glob("*.wav")]
    if not wav_files:
        print("No .wav files found in the selected folder.")
        return

    print(f"\n📂 Processing {len(wav_files)} files in: {folder_path}\n")
    for wav_file in wav_files:
        process_wav_file(wav_file, amplitude_folder, spectrogram_folder,
                         segment_duration=segment_duration,
                         sigma=sigma,
                         threshold=threshold)

# === USER INPUT ===
#folder_path = '/Volumes/ROSE2-SSD/RHV_song_counts/Newest_canaries/RC1_R05_Comp2/1/one_minute_segments'

# Optional parameters:
# - sigma controls smoothing (lower is more sensitive)
# - threshold is the detection cutoff (adjust based on amplitude range)

batch_process_folder(
    folder_path,
    segment_duration=10,   # Length of each panel (in seconds)
    sigma=50,              # Gaussian smoothing
    threshold=100           # Song detection threshold
)


📂 Processing 76 files in: /Volumes/my_own_SSD/UO_stuff/nerve_transections/USA5201/60/one_minute_segments/

Processed segment 1/6 for file one_minute_segment_1 – Song 🎵 Detected
Processed segment 2/6 for file one_minute_segment_1 – Song 🎵 Detected
Processed segment 3/6 for file one_minute_segment_1 – Song 🎵 Detected
Processed segment 4/6 for file one_minute_segment_1 – Song 🎵 Detected
Processed segment 5/6 for file one_minute_segment_1 – Song 🎵 Detected
Processed segment 6/6 for file one_minute_segment_1 – Song 🎵 Detected
✅ Saved: /Volumes/my_own_SSD/UO_stuff/nerve_transections/USA5201/60/one_minute_segments/song_detection_outputs/amplitude_traces/one_minute_segment_1_amplitude_trace.png
✅ Saved: /Volumes/my_own_SSD/UO_stuff/nerve_transections/USA5201/60/one_minute_segments/song_detection_outputs/detected_song_spectrograms/one_minute_segment_1_detected_song_spectrogram.png

Processed segment 1/6 for file one_minute_segment_2 – Song 🎵 Detected
Processed segment 2/6 for file one_minute_s

In [5]:
import numpy as np
import os
from pathlib import Path
from scipy.io import wavfile
from scipy.signal import spectrogram, windows, ellip, filtfilt
from scipy.ndimage import gaussian_filter1d
import matplotlib.pyplot as plt
import tkinter as tk
import json

def get_screen_resolution():
    root = tk.Tk()
    root.withdraw()
    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()
    root.destroy()
    return screen_width / 100, screen_height / 100  # inches

width_inches, height_inches = get_screen_resolution()

def process_wav_file(file_path, amplitude_folder, spectrogram_folder, file_lists=None,
                     segment_duration=10, low_cut=500, high_cut=8000, sigma=100, threshold=0.05):
    try:
        base_name = Path(file_path).stem
        samplerate, data = wavfile.read(file_path)
        if data.ndim > 1:
            data = data.mean(axis=1)

        nyquist = samplerate / 2
        wp = [low_cut / nyquist, high_cut / nyquist]
        b, a = ellip(5, 0.2, 40, wp, btype='band')
        data = filtfilt(b, a, data)

        duration_seconds = data.shape[0] / samplerate
        segment_length_samples = int(segment_duration * samplerate)
        num_segments = int(np.ceil(duration_seconds / segment_duration))

        amplitude_fig_path = os.path.join(amplitude_folder, f"{base_name}_amplitude_trace.png")
        spectrogram_fig_path = os.path.join(spectrogram_folder, f"{base_name}_detected_song_spectrogram.png")

        fig1, axs_amp = plt.subplots(num_segments, 1, figsize=(width_inches, height_inches), sharex=True, gridspec_kw={'hspace': 0.0})
        fig2, axs_spec = plt.subplots(num_segments, 1, figsize=(width_inches, height_inches), sharex=True, gridspec_kw={'hspace': 0.0})
        if num_segments == 1:
            axs_amp = [axs_amp]
            axs_spec = [axs_spec]

        # Get red line timestamps if JSON is provided
        red_lines_sec = []
        if file_lists and f"{base_name}.wav" in file_lists:
            red_lines_sec = [entry["start_sample"] / samplerate for entry in file_lists[f"{base_name}.wav"]]

        for i in range(num_segments):
            start_sample = i * segment_length_samples
            end_sample = start_sample + segment_length_samples
            segment_data = np.zeros(segment_length_samples, dtype=data.dtype)
            if start_sample < data.shape[0]:
                segment_data[:max(0, min(segment_length_samples, data.shape[0] - start_sample))] = data[start_sample:end_sample]

            f, t, Sxx = spectrogram(
                segment_data,
                fs=samplerate,
                window=windows.gaussian(2048, std=2048/8),
                nperseg=2048,
                noverlap=(2048 - 119)
            )

            Sxx_log = 10 * np.log10(Sxx + np.finfo(float).eps)
            Sxx_log_clipped = np.clip(Sxx_log, a_min=3, a_max=None)
            Sxx_log_normalized = (Sxx_log_clipped - np.min(Sxx_log_clipped)) / (np.max(Sxx_log_clipped) - np.min(Sxx_log_clipped))
            Sxx_log_normalized = np.power(Sxx_log_normalized, 0.7)

            freq_mask = (f >= low_cut) & (f <= high_cut)
            Sxx_band = Sxx[freq_mask, :]
            amplitude_trace = np.sum(Sxx_band, axis=0)
            smoothed_trace = gaussian_filter1d(amplitude_trace, sigma=sigma)
            is_song = smoothed_trace > threshold

            axs_amp[i].plot(t, amplitude_trace, color='gray', alpha=0.5, label='Original')
            axs_amp[i].plot(t, smoothed_trace, color='black', label=f'Smoothed (σ={sigma})')
            axs_amp[i].axhline(threshold, color='red', linestyle='--', linewidth=1, label=f'Threshold = {threshold}')
            axs_amp[i].fill_between(t, smoothed_trace, threshold, where=is_song, interpolate=True,
                                    color='gold', alpha=0.4, label='Detected Song' if i == 0 else None)

            if red_lines_sec:
                for x in red_lines_sec:
                    if start_sample / samplerate <= x < end_sample / samplerate:
                        axs_amp[i].axvline(x - (start_sample / samplerate), color='red', linestyle='-', linewidth=1)

            axs_amp[i].set_ylabel('Amplitude')
            if i == 0:
                axs_amp[i].legend(loc='upper right', fontsize=8)
            if i == num_segments - 1:
                axs_amp[i].set_xlabel('Time [sec]')
                axs_amp[i].set_xticks(np.linspace(0, segment_duration, 5))

            axs_spec[i].imshow(Sxx_log_normalized, aspect='auto', origin='lower',
                               extent=[0, segment_duration, f.min(), f.max()], cmap='binary')
            axs_spec[i].fill_between(t, f.min(), f.max(), where=is_song, color='gold', alpha=0.3)

            if red_lines_sec:
                for x in red_lines_sec:
                    if start_sample / samplerate <= x < end_sample / samplerate:
                        axs_spec[i].axvline(x - (start_sample / samplerate), color='red', linestyle='-', linewidth=1)

            axs_spec[i].set_ylabel('Freq [Hz]')
            if i == num_segments - 1:
                axs_spec[i].set_xlabel('Time [sec]')
                axs_spec[i].set_xticks(np.linspace(0, segment_duration, 5))

            print(f"Processed segment {i+1}/{num_segments} for file {base_name} – Song {'🎵 Detected' if np.any(is_song) else '⚪ Not Detected'}")

        fig1.suptitle(f'{base_name} – Amplitude Traces with Song Detection', fontsize=14)
        fig2.suptitle(f'{base_name} – Spectrogram with Detected Song', fontsize=14)

        fig1.tight_layout()
        fig2.tight_layout()

        fig1.savefig(amplitude_fig_path, dpi=300)
        fig2.savefig(spectrogram_fig_path, dpi=300)
        plt.close(fig1)
        plt.close(fig2)

        print(f"✅ Saved: {amplitude_fig_path}")
        print(f"✅ Saved: {spectrogram_fig_path}\n")

    except Exception as e:
        print(f"❌ Error processing {file_path}: {e}")

def batch_process_folder(folder_path, json_path=None, segment_duration=10, sigma=100, threshold=0.05):
    output_folder = os.path.join(folder_path, "song_detection_outputs")
    amplitude_folder = os.path.join(output_folder, "amplitude_traces")
    spectrogram_folder = os.path.join(output_folder, "detected_song_spectrograms")
    os.makedirs(amplitude_folder, exist_ok=True)
    os.makedirs(spectrogram_folder, exist_ok=True)

    file_lists = None
    # 1. Look for file_lists.json inside folder_path
    default_json = os.path.join(folder_path, "/one_minute_segments/file_lists.json")
    if os.path.exists(default_json):
        with open(default_json, 'r') as f:
            file_lists = json.load(f)
        print(f"📄 Loaded file list from: {default_json}")
    # 2. Otherwise, try user-supplied json_path
    elif json_path and os.path.exists(json_path):
        with open(json_path, 'r') as f:
            file_lists = json.load(f)
        print(f"📄 Loaded file list from: {json_path}")
    else:
        print("ℹ️ No valid .json file found – skipping red lines.")

    wav_files = [f for f in Path(folder_path).glob("*.wav")]
    if not wav_files:
        print("No .wav files found in the selected folder.")
        return

    print(f"\n📂 Processing {len(wav_files)} files in: {folder_path}\n")
    for wav_file in wav_files:
        process_wav_file(wav_file, amplitude_folder, spectrogram_folder,
                         file_lists=file_lists,
                         segment_duration=segment_duration,
                         sigma=sigma,
                         threshold=threshold)

# === USER INPUT ===
#
#json_path = '/Users/mirandahulsey-vincent/Documents/allPythonCode/BYOD_class/data_inputs/USA5483_sample_songs/216_sample/one_minute_segments/file_lists.json'
json_path = None
# You can set json_path = None if you don't want to provide a backup path

batch_process_folder(
    folder_path,
    json_path=json_path,
    segment_duration=10,
    sigma=50,
    threshold=800
)


ℹ️ No valid .json file found – skipping red lines.

📂 Processing 76 files in: /Volumes/my_own_SSD/UO_stuff/nerve_transections/USA5201/60/one_minute_segments/

Processed segment 1/6 for file one_minute_segment_1 – Song ⚪ Not Detected
Processed segment 2/6 for file one_minute_segment_1 – Song ⚪ Not Detected
Processed segment 3/6 for file one_minute_segment_1 – Song ⚪ Not Detected
Processed segment 4/6 for file one_minute_segment_1 – Song 🎵 Detected
Processed segment 5/6 for file one_minute_segment_1 – Song 🎵 Detected
Processed segment 6/6 for file one_minute_segment_1 – Song 🎵 Detected
✅ Saved: /Volumes/my_own_SSD/UO_stuff/nerve_transections/USA5201/60/one_minute_segments/song_detection_outputs/amplitude_traces/one_minute_segment_1_amplitude_trace.png
✅ Saved: /Volumes/my_own_SSD/UO_stuff/nerve_transections/USA5201/60/one_minute_segments/song_detection_outputs/detected_song_spectrograms/one_minute_segment_1_detected_song_spectrogram.png

Processed segment 1/6 for file one_minute_segment_